# Jupyter notebook for taking the BLASTx output (transcriptome from library01 against a blastdb of dinoflagellate) to GOslim

### from Steven's notebook: https://github.com/sr320/nb-2018/blob/master/C_virginica/83-blast-2-slim.ipynb

### Need three files:
1) blastout file in format -6 (http://owl.fish.washington.edu/scaphapoda/grace/PCSGA-2019/Cb_v1_blastx_uniprot2864_imac.tab)       
2) Uniprot GO annotation file (340M) from : http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted       
3) GOslim file from : http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/Go-GOslim.sorted

In [1]:
pwd

'/Users/graciecrandall/Documents/GitHub/project-crab/notebooks'

In [2]:
wd = "/Users/graciecrandall/Documents/GitHub/project-crab/analyses/"

In [3]:
cd {wd}

/Users/graciecrandall/Documents/GitHub/project-crab/analyses


In [4]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2314k  100 2314k    0     0  3244k      0 --:--:-- --:--:-- --:--:-- 3549k


In [5]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340M  100  340M    0     0  16.0M      0  0:00:21  0:00:21 --:--:-- 20.4M  0:00:33  0:00:03  0:00:30 10.2M


In [7]:
!curl -O http://owl.fish.washington.edu/scaphapoda/grace/PCSGA-2019/Cb_v1_blastx_uniprot2864_imac.tab

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3384k  100 3384k    0     0  6033k      0 --:--:-- --:--:-- --:--:-- 6164k


In [8]:
# check if files are in the directory
!ls

Cb_v1_blastx_uniprot2864_imac.tab master-qubit.xlsx
GO-GOslim.sorted                  qubit_results-for-libraries.csv
crab-GOslim.csv                   readme.md
hemat-blastx-output.tab           sample_table.csv
master-qubit-sample.csv           uniprot-SP-GO.sorted


## Set blastout file as variable

In [9]:
blastout="Cb_v1_blastx_uniprot2864_imac.tab"

In [10]:
!head -2 $blastout

TRINITY_DN21464_c0_g1_i1	tr|A0A1Q9E613|A0A1Q9E613_SYMMI	38.298	611	346	10	801	2570	567	1167	6.69e-116	382
TRINITY_DN21426_c0_g2_i1	tr|A0A1Q9DMD0|A0A1Q9DMD0_SYMMI	46.383	470	235	3	126	1484	34	503	2.34e-134	406


In [11]:
# convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [12]:
!head -2 _blast-sep.tab

TRINITY_DN21464_c0_g1_i1	tr	A0A1Q9E613	A0A1Q9E613_SYMMI	38.298	611	346	10	801	2570	567	1167	6.69e-116	382
TRINITY_DN21426_c0_g2_i1	tr	A0A1Q9DMD0	A0A1Q9DMD0_SYMMI	46.383	470	235	3	126	1484	34	503	2.34e-134	406


In [13]:
#reducing number of columns and sorting 
!awk -v OFS='\t' '{print $3, $1, $13}' <_blast-sep.tab | sort \
> _blast-sort.tab

In [14]:
!head -2 _blast-sort.tab

A0A075C407	TRINITY_DN5199_c0_g1_i1	1.04e-47
A0A075C407	TRINITY_DN5199_c0_g1_i2	1.26e-30


In [15]:
#joining blast with uniprot annotation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [16]:
!head _blast-annot.tab

A8I1Q0	TRINITY_DN10831_c0_g1_i1	GO:0005509
A8I1Q0	TRINITY_DN10831_c0_g1_i4	GO:0005509
A8I1Q0	TRINITY_DN1334_c0_g1_i2	GO:0005509
A8I1Q0	TRINITY_DN1334_c0_g1_i3	GO:0005509
A8I1Q0	TRINITY_DN1334_c0_g1_i4	GO:0005509
A8I1Q0	TRINITY_DN1680_c0_g1_i2	GO:0005509
A8I1Q0	TRINITY_DN1680_c0_g1_i3	GO:0005509
A8I1Q0	TRINITY_DN1680_c0_g1_i4	GO:0005509
A8I1Q0	TRINITY_DN21661_c0_g1_i1	GO:0005509
A8I1Q0	TRINITY_DN21661_c0_g1_i1	GO:0005509


## The following is a script modified by Sam White

In [17]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [18]:
!head _blast-GO-unfolded.tab

A8I1Q0	TRINITY_DN10831_c0_g1_i1	GO:0005509
A8I1Q0	TRINITY_DN10831_c0_g1_i4	GO:0005509
A8I1Q0	TRINITY_DN1334_c0_g1_i2	GO:0005509
A8I1Q0	TRINITY_DN1334_c0_g1_i3	GO:0005509
A8I1Q0	TRINITY_DN1334_c0_g1_i4	GO:0005509
A8I1Q0	TRINITY_DN1680_c0_g1_i2	GO:0005509
A8I1Q0	TRINITY_DN1680_c0_g1_i3	GO:0005509
A8I1Q0	TRINITY_DN1680_c0_g1_i4	GO:0005509
A8I1Q0	TRINITY_DN21661_c0_g1_i1	GO:0005509
A8I1Q0	TRINITY_DN21661_c0_g1_i1	GO:0005509


In [22]:
!awk '{print $3,"\t" $2}' _blast-GO-unfolded.tab | gsort -V > _blast-GO-unfolded.sorted

In [23]:
!head _blast-GO-unfolded.sorted

GO:0005509 	TRINITY_DN286_c0_g1_i3
GO:0005509 	TRINITY_DN1334_c0_g1_i2
GO:0005509 	TRINITY_DN1334_c0_g1_i3
GO:0005509 	TRINITY_DN1334_c0_g1_i4
GO:0005509 	TRINITY_DN1680_c0_g1_i2
GO:0005509 	TRINITY_DN1680_c0_g1_i3
GO:0005509 	TRINITY_DN1680_c0_g1_i4
GO:0005509 	TRINITY_DN2638_c0_g1_i2
GO:0005509 	TRINITY_DN2638_c0_g1_i5
GO:0005509 	TRINITY_DN2638_c0_g1_i5


In [24]:
!tr " " "*" < _blast-GO-unfolded.sorted | head

GO:0005509*	TRINITY_DN286_c0_g1_i3
GO:0005509*	TRINITY_DN1334_c0_g1_i2
GO:0005509*	TRINITY_DN1334_c0_g1_i3
GO:0005509*	TRINITY_DN1334_c0_g1_i4
GO:0005509*	TRINITY_DN1680_c0_g1_i2
GO:0005509*	TRINITY_DN1680_c0_g1_i3
GO:0005509*	TRINITY_DN1680_c0_g1_i4
GO:0005509*	TRINITY_DN2638_c0_g1_i2
GO:0005509*	TRINITY_DN2638_c0_g1_i5
GO:0005509*	TRINITY_DN2638_c0_g1_i5


In [25]:
!sed 's/ //g' _blast-GO-unfolded.sorted > _blast-GO-unfolded-nos.sorted

In [26]:
!head _blast-GO-unfolded-nos.sorted

GO:0005509	TRINITY_DN286_c0_g1_i3
GO:0005509	TRINITY_DN1334_c0_g1_i2
GO:0005509	TRINITY_DN1334_c0_g1_i3
GO:0005509	TRINITY_DN1334_c0_g1_i4
GO:0005509	TRINITY_DN1680_c0_g1_i2
GO:0005509	TRINITY_DN1680_c0_g1_i3
GO:0005509	TRINITY_DN1680_c0_g1_i4
GO:0005509	TRINITY_DN2638_c0_g1_i2
GO:0005509	TRINITY_DN2638_c0_g1_i5
GO:0005509	TRINITY_DN2638_c0_g1_i5


In [27]:
!tr " " "*" < _blast-GO-unfolded-nos.sorted | head

GO:0005509	TRINITY_DN286_c0_g1_i3
GO:0005509	TRINITY_DN1334_c0_g1_i2
GO:0005509	TRINITY_DN1334_c0_g1_i3
GO:0005509	TRINITY_DN1334_c0_g1_i4
GO:0005509	TRINITY_DN1680_c0_g1_i2
GO:0005509	TRINITY_DN1680_c0_g1_i3
GO:0005509	TRINITY_DN1680_c0_g1_i4
GO:0005509	TRINITY_DN2638_c0_g1_i2
GO:0005509	TRINITY_DN2638_c0_g1_i5
GO:0005509	TRINITY_DN2638_c0_g1_i5


In [28]:
!head GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P


In [29]:
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded-nos.sorted \
GO-GOslim.sorted | head 

GO:0005509	TRINITY_DN286_c0_g1_i3	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN1334_c0_g1_i2	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN1334_c0_g1_i3	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN1334_c0_g1_i4	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN1680_c0_g1_i2	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN1680_c0_g1_i3	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN1680_c0_g1_i4	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN2638_c0_g1_i2	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN2638_c0_g1_i5	calcium ion binding	other molecular function	F
GO:0005509	TRINITY_DN2638_c0_g1_i5	calcium ion binding	other molecular function	F


In [30]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded-nos.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $2, $4, $5}' \
> Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

      27 Blastquery-GOslim.tab


In [31]:
!head Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

TRINITY_DN286_c0_g1_i3	other molecular function	F
TRINITY_DN1334_c0_g1_i2	other molecular function	F
TRINITY_DN1334_c0_g1_i3	other molecular function	F
TRINITY_DN1334_c0_g1_i4	other molecular function	F
TRINITY_DN1680_c0_g1_i2	other molecular function	F
TRINITY_DN1680_c0_g1_i3	other molecular function	F
TRINITY_DN1680_c0_g1_i4	other molecular function	F
TRINITY_DN2638_c0_g1_i2	other molecular function	F
TRINITY_DN2638_c0_g1_i5	other molecular function	F
TRINITY_DN2638_c0_g1_i6	other molecular function	F
      27 Blastquery-GOslim.tab


## Get P - query and slim

In [38]:
!awk -F"\t" '$3 == "F" { print $1"\t"$2 }' Blastquery-GOslim.tab | sort > Blastquery-GOslim.sorted
!head Blastquery-GOslim.sorted

TRINITY_DN10831_c0_g1_i1	other molecular function
TRINITY_DN10831_c0_g1_i4	other molecular function
TRINITY_DN1334_c0_g1_i2	other molecular function
TRINITY_DN1334_c0_g1_i3	other molecular function
TRINITY_DN1334_c0_g1_i4	other molecular function
TRINITY_DN1680_c0_g1_i2	other molecular function
TRINITY_DN1680_c0_g1_i3	other molecular function
TRINITY_DN1680_c0_g1_i4	other molecular function
TRINITY_DN21661_c0_g1_i1	other molecular function
TRINITY_DN21661_c0_g3_i1	other molecular function


TRINITY_DN286_c0_g1_i3	other molecular function	F
TRINITY_DN1334_c0_g1_i2	other molecular function	F
TRINITY_DN1334_c0_g1_i3	other molecular function	F
TRINITY_DN1334_c0_g1_i4	other molecular function	F
TRINITY_DN1680_c0_g1_i2	other molecular function	F
TRINITY_DN1680_c0_g1_i3	other molecular function	F
TRINITY_DN1680_c0_g1_i4	other molecular function	F
TRINITY_DN2638_c0_g1_i2	other molecular function	F
TRINITY_DN2638_c0_g1_i5	other molecular function	F
TRINITY_DN2638_c0_g1_i6	other molecular function	F
